## Escolha do modelo e referência

O modelo T5 (Text-to-Text Transfer Transformer) oferece a vantagem de tratar diferentes tipos de problemas de PLN, como tradução, resumo, classificação, entre outros, com a mesma estrutura base. A capacidade de T5 de ser fine-tuned em múltiplas tarefas com eficácia reduz a necessidade de criar modelos distintos para cada tarefa específica, o que o torna eficiente e escalável.

Uma das principais vantagens do T5 é seu design unificado e a utilização do pretraining multitarefa em um amplo conjunto de dados, o que lhe confere uma robustez em tarefas variadas. Além disso, o modelo é baseado em transformers, uma arquitetura que já demonstrou ser extremamente eficiente em capturar dependências de longo alcance em sequências de texto, possibilitando maior precisão em tarefas complexas.

### Referência da justificativa

Referência: Raffel, C., Shazeer, N., Roberts, A., Lee, K., Narang, S., Matena, M., Zhou, Y., Li, W., & Liu, P. J. (2020). Exploring the Limits of Transfer Learning with a Unified Text-to-Text Transformer. Journal of Machine Learning Research, 21(140), 1-67

# Importação de bibliotecas

In [ ]:
!pip install pyarrow

API não oficial do Google Translate para realizar traduções automáticas de textos entre diversos idiomas. Para este projeto as bibliotecas padrões de data augmentation trabalhavam com apenas palavras em inglês e por isso utilizamos a biblioteca para traduzir depois do processo de augmentation.

In [ ]:
pip install googletrans==4.0.0-rc1 textblob nltk

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 4.5 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17397 sha256=9dabad6550cca0030f2a78c2c0b55a066430f104f9cb1bb7ee33da80ead4bb5f
  Stored in directory: /root/.cache/pip/wheels/c0/59/9f/7372f0cf70160fe61b528532e1a7c8498c4becd6bcffb022de
Successfully built googletrans
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling chardet-5.2.0:

In [ ]:
# Importar bibliotecas
import time
import psutil
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from googletrans import Translator
import random
import nltk
from nltk.corpus import wordnet

from sklearn.model_selection import train_test_split


import torch

import unittest
from unittest.mock import patch

from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments

from torch.utils.data import Dataset

Biblioteca NLTK para fazer download do Wordnet que é uma grande base de dados lexical para a língua inglesa que agrupa palavras em conjuntos de sinônimos (synsets) e oferece definições curtas e descrições e OMW (Open Multilingual Wordnet), um recurso que estende o WordNet para vários idiomas, fornecendo traduções de synsets em diferentes línguas.

In [ ]:
# Baixar dados do NLTK
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

Biblioteca Transformers da Hugging Face, além de bibliotecas relacionadas para manipulação de dados e otimização de desempenho.

In [ ]:
!pip install transformers datasets torch
!pip install --upgrade pip
!pip install --upgrade datasets pyarrow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.9 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.


  Using cached pip-24.2-py3-none-any.whl.metadata (3.6 kB)
Using cached pip-24.2-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


# Conexão ao Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


 ## Base de dados sem Data Augmentation

### Nesta parte utilizaremos uma base de dados sem Data augmentaion, ou seja sem a adição artificial de novos dados para o treinamento da LLM.

Carregamento da base de dados e seleção das features necessárias para o pré-processamento e utilização na LLM.

In [ ]:
# Carregar a base de dados
df = pd.read_excel('/content/drive/Shareddrives/Grupo_05/baseTratada_v3_20240909.xlsx')

# Supondo que a base tem colunas 'pergunta' e 'resposta'
df = df[['Intencao','Pergunta_Abrev', 'Resposta_Abrev']]

Remover valores nulos

In [ ]:
df = df.dropna()

Conversão dos textos em duas listas em Python

In [ ]:
textos = df['Pergunta_Abrev'].tolist()
respostas = df['Resposta_Abrev'].tolist()

Carregamento de um tokenizer pré-treinado para o modelo T5

In [ ]:
# Carregar o tokenizer
tokenizer = T5Tokenizer.from_pretrained('t5-base')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Divisão das listas de respostas e perguntas (textos e rótulos) em conjuntos de treinamento e teste, sendo 80% para o treinamento e 20% para o teste.

In [ ]:
train_texts, test_texts, train_labels, test_labels = train_test_split(respostas, textos, test_size=0.2, random_state=42)

Esse código tokeniza tanto os textos quanto os rótulos (as perguntas), garantindo que estejam no formato correto para serem passados ao modelo de PLN baseado no T5.

In [ ]:
# Tokenizar os textos de treino e teste corretamente
train_encodings = tokenizer(train_texts, padding=True, truncation=True, max_length=512, return_tensors="pt")
test_encodings = tokenizer(test_texts, padding=True, truncation=True, max_length=512, return_tensors="pt")

train_labels = tokenizer(train_labels, padding=True, truncation=True, max_length=512, return_tensors="pt")
test_labels = tokenizer(test_labels, padding=True, truncation=True, max_length=512, return_tensors="pt")

Definição de uma estrutura personalizada de conjunto de dados para manipular dados de texto em um formato que pode ser facilmente utilizado pelo PyTorch. A classe TextDataset facilita o acesso a dados tokenizados e seus rótulos em qualquer etapa do processo de treinamento, garantindo que eles estejam no formato correto (tensores).

In [ ]:
class TextDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __len__(self):
        return len(self.encodings['input_ids'])
    def __getitem__(self, idx):
        # Aqui garantimos que todas as chaves do encoding estão no formato esperado
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels['input_ids'][idx])
        return item

train_dataset = TextDataset(train_encodings, train_labels)
test_dataset = TextDataset(test_encodings, test_labels)

Configuração de um modelo T5 e especifica como ele deve ser treinado. O modelo é carregado a partir de uma versão pré-treinada, e os argumentos de treinamento definem onde os resultados serão salvos, com que frequência o modelo será avaliado, a taxa de aprendizado, o tamanho do batch e o número de épocas de treinamento.

In [ ]:
model = T5ForConditionalGeneration.from_pretrained('t5-base')

training_args = TrainingArguments(
    output_dir='./results',              # diretório para salvar o modelo
    evaluation_strategy="epoch",         # avaliar a cada época
    learning_rate=5e-5,                  # taxa de aprendizado
    per_device_train_batch_size=8,       # tamanho do batch de treinamento
    per_device_eval_batch_size=8,        # tamanho do batch de avaliação
    num_train_epochs=3,                   # número de épocas
    weight_decay=0.01,                   # regularização L2
)

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Criação de um objeto Trainer para facilitar o treinamento do modelo T5, especificando o modelo a ser treinado, os argumentos de treinamento, e os conjuntos de dados de treinamento e avaliação. Ao chamar trainer.train(), o treinamento é iniciado, permitindo que o modelo aprenda com os dados e melhore seu desempenho ao longo do tempo.

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer
)

trainer.train()

<ipython-input-16-e8a5c29b8a55>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-16-e8a5c29b8a55>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels['input_ids'][idx])


Epoch,Training Loss,Validation Loss
1,No log,0.485092
2,No log,0.397913
3,No log,0.385473


<ipython-input-16-e8a5c29b8a55>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-16-e8a5c29b8a55>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels['input_ids'][idx])


TrainOutput(global_step=231, training_loss=0.7689736106178977, metrics={'train_runtime': 255.2244, 'train_samples_per_second': 7.241, 'train_steps_per_second': 0.905, 'total_flos': 648397428940800.0, 'train_loss': 0.7689736106178977, 'epoch': 3.0})

Medição do desempenho de um modelo após o treinamento.

In [ ]:
trainer.evaluate()

<ipython-input-16-e8a5c29b8a55>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-16-e8a5c29b8a55>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels['input_ids'][idx])


{'eval_loss': 0.3854730427265167,
 'eval_runtime': 5.2673,
 'eval_samples_per_second': 29.427,
 'eval_steps_per_second': 3.797,
 'epoch': 3.0}

Extração de valores de perda durante o treinamento e avaliação do modelo a partir do histórico de logs, e em seguida cria um gráfico que ilustra a perda ao longo dos passos de treinamento. Isso é muito útil para monitorar o desempenho do modelo e verificar se ele está aprendendo adequadamente, permitindo identificar se há overfitting ou se o modelo precisa de ajustes.

In [ ]:
# Acessando o histórico de logs (se você habilitou logging)
logs = trainer.state.log_history

# Extraindo os valores de perda (loss) do histórico de logs
train_loss = [log['loss'] for log in logs if 'loss' in log]
eval_loss = [log['eval_loss'] for log in logs if 'eval_loss' in log]

# Verificando se há dados de perda de treinamento e avaliação
if not train_loss or not eval_loss:
    print("Não há dados de perda para plotar. Verifique se o logging foi habilitado durante o treinamento.")
else:
    # Número de passos de treinamento (considerando o comprimento da lista com mais dados)
    steps = np.arange(1, max(len(train_loss), len(eval_loss)) + 1)

    # Criando o gráfico
    plt.figure(figsize=(12, 6))
    if train_loss:  # Plota a perda de treinamento se houver dados
        plt.plot(steps[:len(train_loss)], train_loss, label='Perda de Treinamento', color='blue')
    if eval_loss:  # Plota a perda de avaliação se houver dados
        plt.plot(steps[:len(eval_loss)], eval_loss, label='Perda de Avaliação', color='orange')

    plt.title('Perda Durante o Treinamento do Modelo')
    plt.xlabel('Passos de Treinamento')
    plt.ylabel('Perda')
    plt.legend()
    plt.grid()
    plt.tight_layout()
    plt.show()

Não há dados de perda para plotar. Verifique se o logging foi habilitado durante o treinamento.


Interação do usuário com o modelo de linguagem, enviando perguntas e recebendo respostas geradas. Ele utiliza a capacidade da GPU (se disponível) para acelerar o processamento, tornando a inferência mais rápida

In [ ]:
# Verifica se a GPU está disponível
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Transfere o modelo para o dispositivo (GPU ou CPU)
model.to(device)

def generate_response(question):
    # Codifica a pergunta e move para o dispositivo correto
    input_ids = tokenizer.encode(question, return_tensors='pt', max_length=512, truncation=True).to(device)

    # Gera a resposta
    output = model.generate(input_ids)

    # Decodifica a resposta
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response

# Exemplo de uso
question = "Como fazer remessas?"
response = generate_response(question)
print(response)

ol teve remessas?


## Data Augmentation na base de dados

O Código realiza as seguintes ações:

1. Conta o número de ocorrências de cada intenção no DataFrame.

2. Identifica quais intenções têm menos de 200 ocorrências.

3. Inicializa um tradutor para gerar novas variações de texto a partir das intenções menos frequentes, utilizando tradução como uma técnica de aumento de dados.

In [ ]:
# Contagem de ocorrências de cada intenção
intent_counts = df['Intencao'].value_counts()

# Definir as intenções que possuem menos de 200 ocorrências
intentions_to_augment = intent_counts[intent_counts < 200].index.tolist()

# Traduzir e reverter a tradução para gerar novas variações
translator = Translator()

As funções do código abaixo fazem:

1. translate_back: Gera uma nova versão de um texto traduzindo-o para outro idioma e de volta, potencialmente alterando sua estrutura.

2. replace_with_synonyms: Substitui palavras por sinônimos para criar variações de frases.

3. shuffle_sentence: Embaralha a ordem das palavras em uma frase, gerando uma nova versão dela.

In [ ]:
def translate_back(text):
    try:
        # Tradução para outro idioma (ex: 'es' para espanhol)
        translated = translator.translate(text, dest='en').text
        # Reversão para o idioma original (ex: 'pt' para português)
        return translator.translate(translated, dest='pt').text
    except Exception as e:
        return text

# Função para substituir uma palavra por um sinônimo
def replace_with_synonyms(text):
    try:
      words = translator.translate(text, dest='en').text
      words = text.split()
      new_sentence = []
      for word in words:
          # Procurar sinônimos
          synonyms = wordnet.synsets(word)
          if synonyms:
              # Escolher um sinônimo aleatório
              synonym = random.choice(synonyms).lemmas()[0].name()
              new_sentence.append(synonym)
          else:
              new_sentence.append(word)
      return translator.translate(' '.join(new_sentence), dest='pt').text

    except Exception as e:
        words = text.split()
        new_sentence = []
        for word in words:
            # Procurar sinônimos
            synonyms = wordnet.synsets(word)
            if synonyms:
                # Escolher um sinônimo aleatório
                synonym = random.choice(synonyms).lemmas()[0].name()
                new_sentence.append(synonym)
            else:
                new_sentence.append(word)
        return ' '.join(new_sentence)

# Função para trocar a ordem das palavras na frase
def shuffle_sentence(text):
    words = text.split()
    random.shuffle(words)
    return ' '.join(words)

A função augment_intentions aumenta o número de informações para melhorar a quantidade de dados disponíveis para o treinamento de modelos de aprendizado de máquina em tarefas de processamento de linguagem natural. Ao garantir que cada intenção tenha pelo menos 200 exemplos, a função ajuda a equilibrar o conjunto de dados, potencialmente melhorando o desempenho do modelo. As técnicas de aumento, como tradução, sinônimos e embaralhamento de palavras, são eficazes para gerar diversidade nos dados sem perder o significado original.

In [ ]:
def augment_intentions(df):
    # Contagem de ocorrências de cada intenção
    intent_counts = df['Intencao'].value_counts()

    # Definir as intenções que possuem menos de 200 ocorrências
    intentions_to_augment = intent_counts[intent_counts < 200].index.tolist()

    # Aumentar o dataset com novas variações das intenções até que todas tenham 200 exemplos
    augmented_data = []

    for intent in intentions_to_augment:
        # Filtrar o DataFrame original para a intenção atual
        intent_texts_question = df[df['Intencao'] == intent]['Pergunta_Abrev'].tolist()
        intent_texts_answer = df[df['Intencao'] == intent]['Resposta_Abrev'].tolist()

        current_count_questions = len(intent_texts_question)
        current_count_answers = len(intent_texts_answer)

        # Calcular quantas variações precisamos gerar para perguntas e respostas
        needed_augmentations_questions = 200 - current_count_questions
        needed_augmentations_answers = 200 - current_count_answers

        # Aumentar perguntas
        while current_count_questions < 200 and needed_augmentations_questions > 0:
            original_text = random.choice(intent_texts_question)  # Escolhe uma frase original aleatoriamente
            augmentation_method = random.choice(['translate', 'synonym', 'shuffle'])

            if augmentation_method == 'translate':
                augmented_text = translate_back(original_text)
            elif augmentation_method == 'synonym':
                augmented_text = replace_with_synonyms(original_text)
            else:
                augmented_text = shuffle_sentence(original_text)

            # Adicionar o texto aumentado com a intenção original
            augmented_data.append({'Intencao': intent, 'Pergunta_Abrev': augmented_text})
            current_count_questions += 1
            needed_augmentations_questions -= 1

        # Aumentar respostas
        while current_count_answers < 200 and needed_augmentations_answers > 0:
            original_text = random.choice(intent_texts_answer)  # Escolhe uma frase original aleatoriamente
            augmentation_method = random.choice(['translate', 'synonym', 'shuffle'])

            if augmentation_method == 'translate':
                augmented_text = translate_back(original_text)
            elif augmentation_method == 'synonym':
                augmented_text = replace_with_synonyms(original_text)
            else:
                augmented_text = shuffle_sentence(original_text)

            # Adicionar o texto aumentado com a intenção original
            augmented_data.append({'Intencao': intent, 'Resposta_Abrev': augmented_text})
            current_count_answers += 1
            needed_augmentations_answers -= 1

    # Criar um DataFrame a partir dos dados aumentados
    augmented_df = pd.DataFrame(augmented_data)

    # Concatenar o DataFrame original com o DataFrame aumentado
    augmented_full_df = pd.concat([df, augmented_df], ignore_index=True)

    return augmented_full_df


A função augment_intentions é chamada com o DataFrame df, que deve conter duas colunas relevantes: Intencao (representando as intenções) e Pergunta_Abrev (representando as perguntas associadas).

In [ ]:
augment_intentions = augment_intentions(df)

Remoção de dados inconsistentes

In [ ]:
augment_intentions = augment_intentions.dropna()
augment_intentions = augment_intentions.drop_duplicates()

Conversão dos textos com augmentação em listas Python

In [ ]:
textos_augment = augment_intentions['Pergunta_Abrev'].tolist()
respostas_augment = augment_intentions['Resposta_Abrev'].tolist()

Divisão das listas de respostas e perguntas (textos e rótulos com augmentação) em conjuntos de treinamento e teste, sendo 80% para o treinamento e 20% para o teste.

In [ ]:
train_texts_augment, test_texts_augment, train_labels_augment, test_labels_augment = train_test_split(respostas_augment, textos_augment, test_size=0.2, random_state=42)

Esse código tokeniza tanto os textos quanto os rótulos com augmentação (as perguntas), garantindo que estejam no formato correto para serem passados ao modelo de PLN baseado no T5.

In [ ]:
# Tokenizar os textos de treino e teste corretamente
train_encodings_augment = tokenizer(train_texts_augment, padding=True, truncation=True, max_length=512, return_tensors="pt")
test_encodings_augment = tokenizer(test_texts_augment, padding=True, truncation=True, max_length=512, return_tensors="pt")

train_labels_augment = tokenizer(train_labels_augment, padding=True, truncation=True, max_length=512, return_tensors="pt")
test_labels_augment = tokenizer(test_labels_augment, padding=True, truncation=True, max_length=512, return_tensors="pt")

Esse código prepara os dados aumentados para serem utilizados no modelo T5, estruturando os exemplos (entradas e saídas) de treinamento e teste no formato necessário para o treinamento do modelo.

In [ ]:
train_dataset_augment = TextDataset(train_encodings_augment, train_labels_augment)
test_dataset_augment = TextDataset(test_encodings_augment, test_labels_augment)

Carregamento do modelo T5-base da Hugging Face e armazenamento na variável model_aug para utilizá-lo posteriormente com os dados aumentados que foram preparados.

In [ ]:
model_aug = T5ForConditionalGeneration.from_pretrained('t5-base')


Configurações que ajustam o comportamento do treinamento, incluindo a frequência de avaliação, a taxa de aprendizado, o tamanho do batch e a regularização.

In [ ]:
training_args_aug = TrainingArguments(
    output_dir='./results',              # diretório para salvar o modelo
    evaluation_strategy="epoch",         # avaliar a cada época
    learning_rate=5e-5,                  # taxa de aprendizado
    per_device_train_batch_size=8,       # tamanho do batch de treinamento
    per_device_eval_batch_size=8,        # tamanho do batch de avaliação
    num_train_epochs=3,                   # número de épocas
    weight_decay=0.01,                   # regularização L2
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Treinamento é realizado com os dados aumentados, que foram criados para melhorar o desempenho do modelo ao lidar com intenções menos frequentes, gerando variações para garantir um melhor balanceamento entre as diferentes classes (intenções).

In [ ]:
trainer_aug = Trainer(
    model=model_aug,
    args=training_args_aug,
    train_dataset=train_dataset_augment,
    eval_dataset=test_dataset_augment,
    tokenizer=tokenizer
)

trainer_aug.train()


<ipython-input-22-e8a5c29b8a55>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-22-e8a5c29b8a55>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels['input_ids'][idx])


Epoch,Training Loss,Validation Loss
1,No log,0.537263
2,No log,0.450795
3,No log,0.439620


<ipython-input-22-e8a5c29b8a55>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-22-e8a5c29b8a55>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels['input_ids'][idx])


TrainOutput(global_step=228, training_loss=0.773596445719401, metrics={'train_runtime': 267.795, 'train_samples_per_second': 6.789, 'train_steps_per_second': 0.851, 'total_flos': 637871496652800.0, 'train_loss': 0.773596445719401, 'epoch': 3.0})

O código usa o modelo treinado para responder a perguntas de forma autônoma, simulando o comportamento de um chatbot.

In [ ]:
# Transfere o modelo para o dispositivo (GPU ou CPU)
model_aug.to(device)

def generate_response(question):
    # Codifica a pergunta e move para o dispositivo correto
    input_ids = tokenizer.encode(question, return_tensors='pt', max_length=512, truncation=True).to(device)

    # Gera a resposta
    output = model_aug.generate(input_ids)

    # Decodifica a resposta
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response

# Exemplo de uso
question = "Oi, tudo bem?"
response = generate_response(question)
print(response)

Oi, tudo apenas?


Avaliação do modelo treinado com augmentação

In [ ]:
trainer_aug.evaluate()

Salvamento do modelo com augmentação

In [ ]:
model.save_pretrained('./t5-fine-tuned')
tokenizer.save_pretrained('./t5-fine-tuned')


('./t5-fine-tuned/tokenizer_config.json',
 './t5-fine-tuned/special_tokens_map.json',
 './t5-fine-tuned/spiece.model',
 './t5-fine-tuned/added_tokens.json')

Esse gráfico visualiza como as intenções foram aumentadas e compara suas contagens antes e depois do processo de data augmentation.

In [ ]:
def plot_intent_counts(original_df, augmented_df):
    # Contagem de ocorrências de cada intenção no DataFrame original
    original_counts = original_df['Intencao'].value_counts()

    # Contagem de ocorrências de cada intenção no DataFrame aumentado
    augmented_counts = augmented_df['Intencao'].value_counts()

    # Criar um DataFrame para facilitar a plotagem
    counts_df = pd.DataFrame({
        'Intencao': original_counts.index,
        'Original': original_counts.values,
        'Aumentado': augmented_counts.reindex(original_counts.index, fill_value=0).values
    })

    # Configurar o gráfico
    plt.figure(figsize=(12, 6))
    counts_df.set_index('Intencao').plot(kind='bar', stacked=True)

    # Adicionar título e rótulos
    plt.title('Contagem de Intenções: Original vs Aumentado')
    plt.xlabel('Intenção')
    plt.ylabel('Contagem')
    plt.xticks(rotation=45)
    plt.legend(title='Tipo', loc='upper left')

    # Mostrar o gráfico
    plt.tight_layout()
    plt.show()